https://levelup.gitconnected.com/how-to-use-python-to-create-custom-star-maps-for-your-next-stargazing-journey-9908b421f30e

In [ ]:
#install the required libraries for the first time 
#!pip install skyfield
#!pip install tzwhere
#!pip install geopy

from datetime import datetime
from geopy import Nominatim
from tzwhere import tzwhere
from pytz import timezone, utc

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Circle

from skyfield.api import Star, load, wgs84
from skyfield.data import hipparcos, mpc, stellarium
from skyfield.projections import build_stereographic_projection
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN

In [ ]:
def load_data():
    # de421 shows position of earth and sun in space
    eph = load('de421.bsp')

    # hipparcos dataset
    with load.open(hipparcos.URL) as f:
        stars = hipparcos.load_dataframe(f)

    # constellation dataset
    url = ('https://raw.githubusercontent.com/Stellarium/stellarium/master'
           '/skycultures/modern_st/constellationship.fab')

    with load.open(url) as f:
        constellations = stellarium.parse_constellations(f)
        
    return eph, stars, constellations
        
# load celestial data
eph, stars, constellations = load_data()

In [ ]:
from geopy.geocoders import Nominatim

def get_coordinates(city_name):
    # Initialize the Nominatim geocoder
    geolocator = Nominatim(user_agent="city_coordinates")

    try:
        # Use the geocoder to get the location information
        location = geolocator.geocode(city_name)

        if location:
            # Extract latitude and longitude
            latitude = location.latitude
            longitude = location.longitude
            return latitude, longitude
        else:
            return None
    except Exception as e:
        print("Error:", str(e))
        return None

# Example usage:
city_name = "New York"
coordinates = get_coordinates(city_name)

if coordinates:
    print(f"Coordinates of {city_name}: Latitude = {coordinates[0]}, Longitude = {coordinates[1]}")
else:
    print(f"Coordinates for {city_name} not found.")

Coordinates of New York: Latitude = 40.7127281, Longitude = -74.0060152


In [17]:
def collect_celestial_data(location, when):
    # Get latitude coordinates
    locator = Nominatim(user_agent='myGeocoder', timeout = 10)
    location = get_coordinates(location)
    
    lat, long = location[0], location[1]
    
    # Convert date string into datetime object
    dt = datetime.strptime(when, '%Y-%m-%d %H:%M')

    # Define datetime and convert to UTC based on location coordinates
    timezone_str = tzwhere.tzwhere().tzNameAt(lat, long)
    local = timezone(timezone_str)
    utc_dt = local.localize(dt, is_dst=None).astimezone(utc)

    # Define observer using location coordinates and current UTC time
    t = load.timescale().from_datetime(utc_dt)
    observer = wgs84.latlon(latitude_degrees=lat, longitude_degrees=long).at(t)

    # An ephemeris on Sun and Earth positions.
    sun = eph['sun']
    earth = eph['earth']
    
    # And the constellation outlines list.
    edges = [edge for name, edges in constellations for edge in edges]
    edges_star1 = [star1 for star1, star2 in edges]
    edges_star2 = [star2 for star1, star2 in edges]

    
    # Define the angle and center the observation location by the angle
    position = observer.from_altaz(alt_degrees=90, az_degrees=0)
    ra, dec, distance = observer.radec()
    center_object = Star(ra=ra, dec=dec)

    # Build the stereographic projection
    center = earth.at(t).observe(center_object)
    projection = build_stereographic_projection(center)
    field_of_view_degrees = 180.0

    # Compute the x and y coordinates based on the projection
    star_positions = earth.at(t).observe(Star.from_dataframe(stars))
    stars['x'], stars['y'] = projection(star_positions)
    
    return stars, edges_star1, edges_star2
    
    
# call the above function with a given location and timestamp
location = 'New York, NY'
when = '2023-12-01 00:00'
#stars, edges_star1, edges_star2 = collect_celestial_data(location, when)

In [25]:
location = 'New York, NY'
when = '2023-12-01 00:00'
locator = Nominatim(user_agent='myGeocoder', timeout = 10)
location = get_coordinates(location)
    
lat, long = location[0], location[1]
    
# Convert date string into datetime object
dt = datetime.strptime(when, '%Y-%m-%d %H:%M')


In [ ]:
local = timezone(timezone_str)
utc_dt = local.localize(dt, is_dst=None).astimezone(utc)
# Define observer using location coordinates and current UTC time
t = load.timescale().from_datetime(utc_dt)
observer = wgs84.latlon(latitude_degrees=lat, longitude_degrees=long).at(t)
# An ephemeris on Sun and Earth positions.
sun = eph['sun']
earth = eph['earth']
    